In [ ]:
import pandas as pd
from cellxgene_mods import CxG_API

**Specify the Published ID of the Collection (not the Revision ID)**

In [ ]:
collection_id = ''

In [ ]:
collection = CxG_API.get_collection(collection_id)
rev_id = collection['revising_in']
rev = CxG_API.get_collection(rev_id)

In [ ]:
should_differ_collection = [
    'collection_id', 'collection_url', 'collection_version_id',
    'created_at', 'revising_in', 'revision_of', 'visibility'
]
should_be_absent = [
    'processing_status'
]

should_differ_dataset = [
    'dataset_version_id','explorer_url','assets','revised_at','citation','processing_status'
]

ont_fields = [
    'assay','cell_type','development_stage','disease',
    'self_reported_ethnicity','sex','tissue','organism'
]

In [ ]:
for k,v in rev.items():
    if k not in collection.keys():
        if k not in should_be_absent:
            print('not present: ' + k)
    elif collection.get(k) != v and k not in should_differ_collection:
        if k == 'datasets':
            diff_props = []
            pub_datasets = {d['dataset_id']: d for d in collection[k]}
            rev_datasets = {d['dataset_id']: d for d in rev[k]}
            comp = {}
            for ds_id,v in rev_datasets.items():
                comp[ds_id] = {'title': v['title']}
                if ds_id not in pub_datasets.keys():
                    comp[ds_id]['in published'] = 'not present'
                else:
                    for prop,rev_val in v.items():
                        if prop not in should_differ_dataset:
                            pub_val = pub_datasets[ds_id].get(prop)
                            if prop in ont_fields:
                                rev_val = [t['label'] for t in rev_val]
                                pub_val = [t['label'] for t in pub_val]
                            if isinstance(rev_val, list) and prop != 'assets':
                                rev_val.sort()
                                pub_val.sort()
                            if pub_val != rev_val:
                                if prop == 'mean_genes_per_cell' and round(rev_val, 5) == round(pub_val, 5):
                                    continue
                                diff_props.append(prop)
                                comp[ds_id][prop + '_REV'] = rev_val
                                comp[ds_id][prop + '_PUB'] = pub_val
        else:
            print('not same: ' + k)
            if k not in ['datasets','publisher_metadata']:
                if k in ['links']:
                    diff_in_pub = [l for l in collection[k] if l not in v]
                    print('--- published: ', diff_in_pub)
                    diff_in_rev = [l for l in v if l not in collection[k]]
                    print('----- revised: ', diff_in_rev)
                else:
                    print('--- published: ', str(collection[k]))
                    print('----- revised: ', v)


comp_df = pd.DataFrame(comp).transpose()
comp_df = comp_df.dropna(subset=[c for c in comp_df.columns if c != 'title'], how='all')
if not comp_df.empty:
    display(comp_df.fillna(''))

for f in diff_props:
    temp = comp_df[(comp_df[f + '_REV'] != comp_df[f + '_PUB']) & (comp_df[f + '_PUB'].isna() == False)]
    for i,row in temp.iterrows():
        print(i + '-' + f)
        p = row[f + '_PUB']
        r = row[f + '_REV']
        only_in_pub = [e for e in p if e not in r]
        only_in_rev = [e for e in r if e not in p]
        print('only in pub:' + ','.join(only_in_pub))
        print('only in rev:' + ','.join(only_in_rev))
        print('---------')